In [ ]:
from google.colab import drive
import zipfile, os, urllib.request, glob, math
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import zipfile, os, urllib.request, glob, math, shutil
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import OneHotEncoder
drive.mount('/content/drive')

random.seed(9)

Mounted at /content/drive


In [ ]:
metadata = pd.read_csv('drive/MyDrive/metadata_processed.csv')
metadata.head()

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,UNK,UNK,UNK,UNK,8,UNK,UNK,UNK,...,-5.0,3,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,1,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,UNK,UNK,UNK,UNK,77,UNK,UNK,UNK,...,-5.0,0,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,UNK,UNK,UNK,UNK,75,UNK,UNK,UNK,...,-5.0,0,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,1,True,True,False,False,True,True,PAT_684_1302_588.png,True


In [ ]:
numerical_vars = ['age', 'diameter_1', 'diameter_2']
categorical_vars = sorted(list(set(metadata.columns.tolist()) - set(numerical_vars) - set(['patient_id', 'lesion_id','img_id', 'biopsed', 'diagnostic'])))

metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2298 entries, 0 to 2297
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   patient_id           2298 non-null   object 
 1   lesion_id            2298 non-null   int64  
 2   smoke                2298 non-null   object 
 3   drink                2298 non-null   object 
 4   background_father    2298 non-null   object 
 5   background_mother    2298 non-null   object 
 6   age                  2298 non-null   int64  
 7   pesticide            2298 non-null   object 
 8   gender               2298 non-null   object 
 9   skin_cancer_history  2298 non-null   object 
 10  cancer_history       2298 non-null   object 
 11  has_piped_water      2298 non-null   object 
 12  has_sewage_system    2298 non-null   object 
 13  fitspatrick          2298 non-null   object 
 14  region               2298 non-null   object 
 15  diameter_1           2298 non-null   f

In [ ]:
metadata_wdp = metadata.drop_duplicates(subset = ['lesion_id']).copy(deep = True)
print('Length of dataset without duplicates is: ', len(metadata_wdp))

Length of dataset without duplicates is:  1641


Χρησιμοποιούμε 5-fold cross validation για την αξιολόγηση του μοντέλου. Ο διαχωρισμός γίνεται στο metadata_wdp ώστε να μοιραστούν όλες οι εικόνες από μία πληγή στο ίδιο fold.

In [ ]:
skf = StratifiedKFold(n_splits=5)
for i, (_, test_idx) in enumerate(skf.split(metadata_wdp['img_id'].tolist(), metadata_wdp['diagnostic'].tolist())):

  print("Fold %d , length of test set: %d " %(i,len(test_idx)))
  to_append_list = [0] * len(metadata_wdp)
  for j in range(len(to_append_list)): #τωρα παρατηρησα οτι θα μπορουσα να διατρεχω το test_idx
    if j in test_idx:
      to_append_list[j] = 1

  column_name = 'Fold_' + str(i)
  metadata_wdp[column_name] = to_append_list

metadata_wdp.head()

Fold 0 , length of test set: 329 
Fold 1 , length of test set: 328 
Fold 2 , length of test set: 328 
Fold 3 , length of test set: 328 
Fold 4 , length of test set: 328 


,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,changed,bleed,elevation,img_id,biopsed,Fold_0,Fold_1,Fold_2,Fold_3,Fold_4
0,PAT_1516,1765,UNK,UNK,UNK,UNK,8,UNK,UNK,UNK,...,False,False,False,PAT_1516_1765_530.png,False,1,0,0,0,0
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,True,True,True,PAT_46_881_939.png,True,1,0,0,0,0
2,PAT_1545,1867,UNK,UNK,UNK,UNK,77,UNK,UNK,UNK,...,False,False,False,PAT_1545_1867_547.png,False,1,0,0,0,0
3,PAT_1989,4061,UNK,UNK,UNK,UNK,75,UNK,UNK,UNK,...,False,False,False,PAT_1989_4061_934.png,False,1,0,0,0,0
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,False,True,True,PAT_684_1302_588.png,True,1,0,0,0,0


Ορίζω συνάρτηση για frequency imputaion. Κάθε άγνωστη τιμή θα πάρει μία έγκυρη τιμή η οποία θα επιλεχθεί πιθανοτικά, αναλόγως τη συχνότητα εμφάνισης στο εκάστοτε πεδίο. Μεγάλο πλήθος της τιμής στο πεδίο σημαίνει μεγάλη πιθανότητα επιλογής. Με αυτό το τρόπο διατηρείται η κατανομή των τιμών του εκάστοτε πεδίου.

In [ ]:
def frequency_imputation(train_dataset, test_dataset, column):

  value_counts = train_dataset[column].value_counts().to_dict()

  if 'UNK' in value_counts:
    del value_counts['UNK']

  sum = 0
  for i in value_counts.items():
    sum += i[1]

  for i in value_counts.items():
    value_counts[i[0]] = i[1]/sum

  replacement_choices = list(value_counts.keys())
  replacement_probabilities = list(value_counts.values())

  for lesion in train_dataset.lesion_id.unique():
    random_value = random.choices(replacement_choices, replacement_probabilities)[0]
    train_dataset.loc[(train_dataset['lesion_id'] == lesion) & (train_dataset[column] == 'UNK'), column] = random_value

  for lesion in test_dataset.lesion_id.unique():
    random_value = random.choices(replacement_choices, replacement_probabilities)[0]
    test_dataset.loc[(test_dataset['lesion_id'] == lesion) & (test_dataset[column] == 'UNK'), column] = random_value

Στη συνέχεια, ορίζω συναρτήσεις για την πρόβλεψη των άγνωστων τιμών στα πεδία grew, changed.

In [ ]:
def predict_missing_cat_values(fit_dataset, use_features ,target_feature, fill_datasets):

  train_df = fit_dataset.loc[fit_dataset[target_feature] != 'UNK'][use_features + [target_feature]].copy(deep = True)

  cat_vars = [cat_f for cat_f in use_features if cat_f in categorical_vars]

  #encode target_feature
  le_target = LabelEncoder()
  le_target.fit(train_df[target_feature])
  train_df[target_feature] = le_target.transform(train_df[target_feature])

  cat_encoders = one_hot_encode_fit(train_df, cat_vars)
  train_df = one_hot_encode_transform(train_df, cat_vars, cat_encoders)

  model = RandomForestClassifier(class_weight='balanced')
  sample_weight = compute_sample_weight('balanced', train_df[target_feature])
  model.fit(train_df.drop(columns = [target_feature]), train_df[target_feature], sample_weight = sample_weight)

  for current_dataset in fill_datasets:

    unknown_indeces = (current_dataset[target_feature] == 'UNK')

    fill_dataset = one_hot_encode_transform(current_dataset.loc[unknown_indeces][use_features], cat_vars, cat_encoders)

    predictions = model.predict(fill_dataset)
    current_dataset.loc[unknown_indeces, [target_feature]] = le_target.inverse_transform(predictions)

def split_numpy_to_cols(dataset, array_encodings, column):

  for cols in range(array_encodings.shape[1]):
      column_data = array_encodings[:, cols]
      dataset[column + '_' + str(cols)] = column_data

  dataset.drop(columns = [column], inplace = True)

def one_hot_encode_fit(fit_dataset, columns):

  cat_encoders = []
  for cat_var in columns:
    unique_values = sorted(fit_dataset[cat_var].unique().tolist())
    cat_encoders.append(unique_values)

  return cat_encoders

def one_hot_encode_transform(transform_dataset, columns, encoders):

  temp_dataset = transform_dataset.copy(deep = True)

  for index in range(len(columns)):

    cat_var = columns[index]
    encoder = encoders[index]

    encoded_values = []
    for i in temp_dataset[cat_var]:

      encoded_value = [0] * len(encoder)
      if i not in encoder:
        encoded_values.append(encoded_value)
        continue

      encoded_value[encoder.index(i)] = 1
      encoded_values.append(np.array(encoded_value))

    array_encodings = np.array(encoded_values)
    split_numpy_to_cols(temp_dataset, array_encodings, cat_var)

  return temp_dataset

Τέλος, δημιουργώ τα τελικά datasets, και κωδικοποιώ τα πεδία αναλόγως.

In [ ]:
def one_hot_encode(train_dataset, test_dataset,column):

  unique_vals = sorted(train_dataset[column].unique().tolist())

  train_encodings = []
  for value in train_dataset[column]:

    item_encoding = [0] * len(unique_vals)
    item_encoding[unique_vals.index(value)] = 1
    train_encodings.append(item_encoding)

  train_dataset[column] = train_encodings

  test_encodings = []
  for value in test_dataset[column]:

    item_encoding = [0] * len(unique_vals)

    if value not in unique_vals:
      test_encodings.append(item_encoding)
      continue

    item_encoding[unique_vals.index(value)] = 1
    test_encodings.append(item_encoding)

  test_dataset[column] = test_encodings

for i in ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4']:
  temp_train_dataset = metadata_wdp.loc[metadata_wdp[i] == 0]
  temp_test_dataset = metadata_wdp.loc[metadata_wdp[i] == 1]

  final_train_dataset = metadata[metadata['lesion_id'].isin(temp_train_dataset['lesion_id'])].copy(deep = True).reset_index(drop = True)
  final_test_dataset = metadata[metadata['lesion_id'].isin(temp_test_dataset['lesion_id'])].copy(deep = True).reset_index(drop = True)

  print("ABOUT", i)
  print("Length of train_dataset", len(final_train_dataset))
  print("Length of test_dataset", len(final_test_dataset))

  #scale numerical vars
  for j in numerical_vars:

    fit_data = np.array(final_train_dataset.loc[final_train_dataset[j] != -5][j]).reshape(-1,1)
    print("For %s variable, gength of NaN(-5) values: %d" % (j,len(final_train_dataset.loc[final_train_dataset[j] == -5])))

    scaler = StandardScaler()
    scaler.fit(np.array(fit_data))

    final_train_dataset.loc[final_train_dataset[j] == -5, [j]] = scaler.mean_
    final_test_dataset.loc[final_test_dataset[j] == -5, [j]] = scaler.mean_

    final_train_dataset[j] = scaler.transform(np.array(final_train_dataset[j]).reshape(-1,1)).tolist()
    final_test_dataset[j] = scaler.transform(np.array(final_test_dataset[j]).reshape(-1,1)).tolist()

  #impute categorical_features except grew, changed.
  for j in list(set(categorical_vars) - set(['grew', 'changed'])):
    frequency_imputation(final_train_dataset, final_test_dataset, j)

  for j in ['grew', 'changed']:

    print("Distribution of " + j + " category in train_set before imputation:")
    print(final_train_dataset[j].value_counts())

    predict_missing_cat_values(final_train_dataset,['bleed', 'itch', 'elevation', 'hurt', 'region'], j,[final_train_dataset, final_test_dataset])

    print("Distribution of " + j + " category in train_set after imputation:")
    print(final_train_dataset[j].value_counts())
    print("")

  #encode categorical variables
  for j in categorical_vars:
    one_hot_encode(final_train_dataset, final_test_dataset, j)

  final_train_dataset.to_csv('drive/MyDrive/e_train_dataset_app_' + i + '.csv',index = False)
  final_test_dataset.to_csv('drive/MyDrive/e_test_dataset_app_' + i + '.csv',index = False)

final_train_dataset.head()

ABOUT Fold_0
Length of train_dataset 1751
Length of test_dataset 547
For age variable, gength of NaN(-5) values: 0
For diameter_1 variable, gength of NaN(-5) values: 605
For diameter_2 variable, gength of NaN(-5) values: 605
Distribution of grew category in train_set before imputation:
False    736
True     679
UNK      336
Name: grew, dtype: int64
Distribution of grew category in train_set after imputation:
True     947
False    804
Name: grew, dtype: int64

Distribution of changed category in train_set before imputation:
False    1290
UNK       331
True      130
Name: changed, dtype: int64
Distribution of changed category in train_set after imputation:
False    1416
True      335
Name: changed, dtype: int64

ABOUT Fold_1
Length of train_dataset 1787
Length of test_dataset 511
For age variable, gength of NaN(-5) values: 0
For diameter_1 variable, gength of NaN(-5) values: 621
For diameter_2 variable, gength of NaN(-5) values: 621
Distribution of grew category in train_set before imput

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,"[1, 0]","[1, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",[-3.3403481791993817],"[0, 1]","[1, 0]","[0, 1]",...,[0.0],3,"[1, 0]","[1, 0]","[1, 0]","[1, 0]","[1, 0]","[1, 0]",PAT_1516_1765_530.png,False
1,PAT_46,881,"[1, 0]","[1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",[-0.3461716309690832],"[1, 0]","[1, 0]","[0, 1]",...,[-0.6789002494990505],1,"[0, 1]","[0, 1]","[1, 0]","[0, 1]","[0, 1]","[0, 1]",PAT_46_881_939.png,True
2,PAT_1545,1867,"[1, 0]","[0, 1]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",[1.0553578171387161],"[0, 1]","[1, 0]","[1, 0]",...,[0.0],0,"[0, 1]","[1, 0]","[1, 0]","[1, 0]","[1, 0]","[1, 0]",PAT_1545_1867_547.png,False
3,PAT_1989,4061,"[1, 0]","[0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",[0.9279460491289162],"[0, 1]","[1, 0]","[0, 1]",...,[0.0],0,"[0, 1]","[1, 0]","[1, 0]","[1, 0]","[1, 0]","[1, 0]",PAT_1989_4061_934.png,False
4,PAT_684,1302,"[1, 0]","[0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",[1.182769585148516],"[1, 0]","[0, 1]","[0, 1]",...,[-0.6789002494990505],1,"[0, 1]","[0, 1]","[1, 0]","[1, 0]","[0, 1]","[0, 1]",PAT_684_1302_588.png,True


Κατεβάζω τα δεδομένα.

In [ ]:
urllib.request.urlretrieve("https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/zr7vgbcyr2-1.zip", "imgs.zip")

('imgs.zip', <http.client.HTTPMessage at 0x78a1cdd75870>)

Unzipping.

In [ ]:
with zipfile.ZipFile('imgs.zip', 'r') as zip_ref:
  zip_ref.extractall('imgs_folder')

print("Done with unzipping...")

!ls imgs_folder/images

Done with unzipping...
imgs_part_1.zip  imgs_part_2.zip  imgs_part_3.zip


Κατεβάζω τα δεδομένα και τα αποθηκεύω, ανάλογα με το fold.
Αυτόν τον κώδικα μέχρι το τέλος, τον τρέχω 5 φορές, μία φορά για κάθε fold. Θα μπορούσα να κατεβάζω τα δεδομένα μία φορά, και μετά για κάθε fold να διαχωρίζω τα δεδομένα αναλόγως. Ωστόσο, αυτό προυποθέτει ότι στα notebooks TrainModelApp και TrainModelMetadataApp, θα πρέπει να διαχωρίζω τα δεδομένα στις κλάσεις που ανήκουν, το οποίο καταναλώνει περισσότερο χρόνο και άρα πολλές υπολογιστικές μονάδες. Επομένως, τρέχω αυτόν τον κώδικα για να διαχωρίζω τα δεδομένα και μετά ξεκινώ το training.


In [ ]:
fold_number = 4

final_train_dataset = pd.read_csv('drive/MyDrive/e_train_dataset_app_Fold_' + str(fold_number) + '.csv')
final_test_dataset = pd.read_csv('drive/MyDrive/e_test_dataset_app_Fold_' + str(fold_number) + '.csv')
print(len(final_train_dataset))
print(len(final_test_dataset))

1929
369


Το zip file περιέχει τρία zip, όπου βρίσκονται οι εικόνες και ένα metadata.csv(αυτό που χρησιμοποιώ σε αυτό το notebook). Μαζεύω όλες τις εικόνες σε ένα φάκελο (images).

In [ ]:
for i in ['imgs_folder/images/imgs_part_1.zip', 'imgs_folder/images/imgs_part_2.zip', 'imgs_folder/images/imgs_part_3.zip']:
  with zipfile.ZipFile(i, 'r') as zip_ref:
    zip_ref.extractall()

os.makedirs('images')

dest_folder = 'images'
for src_folder in ['imgs_part_1', 'imgs_part_2', 'imgs_part_3']:
  file_list = os.listdir(src_folder)
  print("Found %d files in folder: %s" %(len(file_list),src_folder))
  for filename in file_list:
    src_path = os.path.join(src_folder,filename)
    shutil.copy(src_path, dest_folder)

!ls images | wc -l

Found 911 files in folder: imgs_part_1
Found 659 files in folder: imgs_part_2
Found 728 files in folder: imgs_part_3
2298


Διαχωρισμός των εικόνων ανάλογα με το split και την κλάση στην οποία ανήκουν.

In [ ]:
diagnostics_list = ['ACK', 'BCC', 'MEL', 'NEV', 'SCC', 'SEK']

def store_to_corresponding_folder(src_path_prefix,dest_path_prefix,current_dataset):

  #create folder for each dataset
  if os.path.exists(dest_path_prefix):
    print("Folder %s already exists..." %dest_path_prefix)
    return
  os.makedirs(dest_path_prefix)

  #create subfolder for each category...
  for i in diagnostics_list:
    folder_path = os.path.join(dest_path_prefix,i)
    os.makedirs(folder_path)

  #store each image to corresponding folder
  for index, row in current_dataset.iterrows():
    category_label = diagnostics_list[row['diagnostic']]
    img_name = row['img_id']

    src_path = os.path.join(src_path_prefix,img_name)
    dest_path = os.path.join(dest_path_prefix,category_label)
    shutil.copy(src_path,dest_path)

  #print number of elements in each folder
  print("For " + dest_path_prefix + ': ')
  for i in diagnostics_list:
    category_folder = os.path.join(dest_path_prefix,i)
    files_no = len(glob.glob(category_folder + '/*.png'))
    print(i + ': ' + str(files_no))

store_to_corresponding_folder('images','train_imgs', final_train_dataset)
store_to_corresponding_folder('images','test_imgs', final_test_dataset)

For train_imgs: 
ACK: 607
BCC: 715
MEL: 45
NEV: 205
SCC: 160
SEK: 197
For test_imgs: 
ACK: 123
BCC: 130
MEL: 7
NEV: 39
SCC: 32
SEK: 38


Αποθήκευση των εικόνων στο drive.

In [ ]:
prefix = 'drive/MyDrive/'

for i in ['train_imgs', 'test_imgs']:
  for j in ['ACK', 'BCC', 'MEL', 'NEV', 'SCC', 'SEK']:
    name = i + '_' + j
    output_filename = prefix + name
    dir_name = i + '/' + j
    shutil.make_archive(output_filename, 'zip', dir_name)